In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection
from blimp.processing.quantify import quantify

24-Oct-24 17:02:10 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
intensity_image_dir = Path("/srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/OME-TIFF-MIP")
label_image_dir = Path("/srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/SEGMENTATION")

intensity_image = AICSImage(intensity_image_dir / "Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")
intron_label_image = AICSImage(label_image_dir / "intron_in_nucleus_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")
nuclei_label_image = AICSImage(label_image_dir / "nuclei_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")
intron_nuclei_labels_label_image = AICSImage(label_image_dir / "nuclei_intron_masked_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")

24-Oct-24 17:02:16 - bfio.bfio.BioReader - DEBUG    - Starting the backend...
24-Oct-24 17:02:16 - bfio.backends.PythonReader - DEBUG    - __init__(): Initializing _rdr (tifffile.TiffFile)...
24-Oct-24 17:02:16 - bfio.backends.PythonReader - DEBUG    - read_metadata(): Reading metadata...
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 256 ImageWidth @24 LONG @36 = 2304
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 257 ImageLength @44 LONG @56 = 2304
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 258 BitsPerSample @64 SHORT @76 = 16
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 259 Compression @84 SHORT @96 = ADOBE_DEFLATE
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 262 PhotometricInterpretation @104 SHORT @116 = MINISBLACK
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 270 ImageDescription @124 ASCII[843] @312 = <OME xmlns="http://www.open
24-Oct-24 17:02:16 - bfio.backends - DEBUG    - TiffTag 273 StripOffsets @144 LONG8[165] 

In [6]:
from blimp.utils import convert_image_dtype, concatenate_images
all_channels = concatenate_images([convert_image_dtype(nuclei_label_image,np.int32),
                    intron_label_image],axis='C',order='append')

24-Oct-24 17:02:44 - fsspec.local - DEBUG    - open file: /srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/SEGMENTATION/nuclei_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff
24-Oct-24 17:02:44 - fsspec.local - DEBUG    - open file: /srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/SEGMENTATION/nuclei_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff
24-Oct-24 17:02:44 - blimp.utils - DEBUG    - Converting array from uint16 to <class 'numpy.int32'>
24-Oct-24 17:02:44 - fsspec.local - DEBUG    - open file: /srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/SEGMENTATION/intron_in_nucleus_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff
24-Oct-24 17:02:44 - blimp.utils - DEBUG    - Concatenating by appending, on axis C


In [8]:
?quantify

Signature:
quantify(
    intensity_image: aicsimageio.aics_image.AICSImage,
    label_image: aicsimageio.aics_image.AICSImage,
    measure_objects: Union[int, str, List[Union[int, str]], NoneType] = None,
    parent_object: Union[int, str, NoneType] = None,
    aggregate: Optional[bool] = False,
    timepoint: Optional[int] = None,
    intensity_channels: Union[int, str, List[Union[int, str]], NoneType] = None,
    texture_channels: Union[int, str, List[Union[int, str]], NoneType] = None,
    texture_objects: Union[int, str, List[Union[int, str]], NoneType] = None,
    texture_scales: list = [1, 3],
) -> Union[pandas.core.frame.DataFrame, List[pandas.core.frame.DataFrame]]
Docstring:
Quantify all channels in an image relative to a matching segmentation
label image.

Parameters
----------
intensity_image
    intensity image (possibly 5D: "TCZYX")
label_image
    label image (possibly 5D: "TCZYX")
timepoint
    which timepoint should be segmented (optional,
    default None will segment 

In [13]:
features_non_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=False)

24-Oct-24 17:08:07 - blimp.utils - INFO     - Channel names unique: ['Nuclei', 'Intron']
24-Oct-24 17:08:07 - blimp.utils - WARNING  - Channel names not unique, renaming to ['647_0', '405_1', '561_2', '405_3']
24-Oct-24 17:08:07 - blimp.processing.quantify - INFO     - ``measure_objects`` =  ['Nuclei', 'Intron']
24-Oct-24 17:08:07 - blimp.processing.quantify - INFO     - ``texture_objects`` =  ['Nuclei', 'Intron']
24-Oct-24 17:08:07 - blimp.processing.quantify - INFO     - ``parent_object`` =  Nuclei
24-Oct-24 17:08:07 - blimp.processing.quantify - DEBUG    - Quantifying Nuclei
24-Oct-24 17:08:07 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.
24-Oct-24 17:08:15 - blimp.processing.quantify - DEBUG    - Quantifying Intron
24-Oct-24 17:08:15 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.


In [14]:
len(features_non_aggregated)

2

In [10]:
features_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=True)

24-Oct-24 17:06:12 - blimp.utils - INFO     - Channel names unique: ['Nuclei', 'Intron']
24-Oct-24 17:06:12 - blimp.utils - WARNING  - Channel names not unique, renaming to ['647_0', '405_1', '561_2', '405_3']
24-Oct-24 17:06:12 - blimp.processing.quantify - INFO     - ``measure_objects`` =  ['Nuclei', 'Intron']
24-Oct-24 17:06:12 - blimp.processing.quantify - INFO     - ``texture_objects`` =  ['Nuclei', 'Intron']
24-Oct-24 17:06:12 - blimp.processing.quantify - INFO     - ``parent_object`` =  Nuclei
24-Oct-24 17:06:12 - blimp.processing.quantify - DEBUG    - Quantifying Nuclei
24-Oct-24 17:06:12 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.
24-Oct-24 17:06:21 - blimp.processing.quantify - DEBUG    - Quantifying Intron
24-Oct-24 17:06:21 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.


In [16]:
features_aggregated.columns

Index(['label', 'Nuclei_centroid_0', 'Nuclei_centroid_1', 'Nuclei_area',
       'Nuclei_area_convex', 'Nuclei_axis_major_length',
       'Nuclei_axis_minor_length', 'Nuclei_eccentricity', 'Nuclei_extent',
       'Nuclei_feret_diameter_max',
       ...
       'Intron_perimeter_crofton_min', 'Intron_perimeter_crofton_max',
       'Intron_perimeter_crofton_std', 'Intron_perimeter_crofton_median',
       'Intron_solidity_mean', 'Intron_solidity_min', 'Intron_solidity_max',
       'Intron_solidity_std', 'Intron_solidity_median', 'Intron_count'],
      dtype='object', length=829)